In [1]:
## load trained q-learning agent.
import pickle
import random
import time
from environment import Environment
from mastermind import Mastermind
from agent import Agent
import pygame
import sys
from random import randint
import socket
import config
import numpy as np
import itertools
import random
from collections import Counter

with open('learned_q_agent.pkl', 'rb') as f:
    q_agent = pickle.load(f)
pygame.init()
env = Mastermind(human=True)
pygame.display.set_caption('GET SET GO PLAYER1!')

def parse_combi(combi_string,act_table):
    
    action_num = int(combi_string[:len(combi_string)-2])
    
        
    return act_table[action_num]
def submit(value,ec):
        try:
            a,b = zip(*[(a,b) for a,b in zip(ec,value) if a!=b])
            a   = list(a)
        except: return 4,0
        for i in b:
            try: a.remove(i)
            except: continue
        return 4-len(b),len(b)-len(a)


def interactive_play(agent):
    player1_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    player1_socket.bind(("localhost", 12347))
    player1_socket.listen(1)
    
    print("Waiting for player 2 to connect...")
    try:
        player2_socket, addr = player1_socket.accept()
        print("Player 2 connected!")
    except ConnectionRefusedError:
        print("Error: Connection to player 2 refused. Make sure player 2 is running.")
    
    player2_socket.send("Starting1".encode())
    
    secret = ""
    flag=0
    
    try:
        while True:
            for ev in pygame.event.get():
                #  this if is working
                if ev.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                # this below is when ever user presses anything on the ui
                elif ev.type == pygame.MOUSEBUTTONDOWN:
                    # this below is for input
                    if env.palet_rect.collidepoint(ev.pos) and len(secret) < 4:
                        x, y = ev.pos
                        x -= env.palet_rect.x
                        y -= env.palet_rect.y
                        try:
                            if env.palet_mask.get_at((x, y)):
                                secret = secret+str(x // 30)
                                env.screen.blit(env.palette, (80 + (len(secret) - 1) * 30, 20),
                                                (x // 30 * 30, 0, 30, 30))
                                pygame.display.flip()
                        except:
                            print("except")
                            continue
                    # this is for reset
                    elif env.reset_rect.collidepoint(ev.pos):
                        secret = ""
                        pygame.display.set_caption('GET SET GO!')
                        flag=0
                        env.screen.blit(env.mmbg, (0, 0))
                        env.screen.blit(env.palette, env.palet_rect)
                        pygame.display.flip()    
                    # this is for go where evaluation should be added
                    elif env.go_rect.collidepoint(ev.pos):
                        if (len(secret) < 4):
                                raise ValueError("please choose 4 numbers")
                        if flag==0:    
                            l=[str(int(i)) for i in list(secret)]
                            player2_socket.send(''.join(l).encode())
                            secret2 = int(player2_socket.recv(1024).decode())
                            print(f"Player 2's secret number: {secret2}")
                            flag=1
                    
                        
                        
    
                        ready = True
                        # Deploy AI agent function here
    
                        agent.reset_possible_states() 
                        env1 = Environment(secret)
                        for ligne in range(6):
                            guess = agent.get_best_action() # prediction = parse_combi(ans[0], env.list_action)
                            # feedback = env.feedback(prediction)
                            for i in range(4):
                                r = env.screen.blit(env.palette, ((i + 1) * 30 + 20, ligne * 35 + 70),
                                                    ((int(guess[i])) * 30, 0, 30, 30))
                                pygame.display.update(r)
                            g="g"+str(guess)
                            player2_socket.send(g.encode())
                            fb = player2_socket.recv(1024).decode()
                            feedback=int(fb[0]), int(fb[1])
                            print(f"{secret}, {guess}, {fb[0]}, {fb[1]}")
                            agent.restrict_possible_states(guess, feedback)
                            placed, misplaced = int(fb[0]),int(fb[1])
                            for e, c in enumerate([(255, 0, 0)] * placed + [(255, 255, 255)] * misplaced):
                                r = pygame.draw.circle(env.screen, c, (190 + e * 10, ligne * 35 + 85), 2, 0)
                                pygame.display.update(r)
                            time.sleep(1)
                            if secret == guess:
                                player2_socket.send("Guessed1".encode())
                                pygame.display.set_caption('WON!')
                                break
                            else:
                                player2_socket.send("Try1".encode())
                            message=player2_socket.recv(1024).decode()
                            if message[0]=="g":
                                guess2= message[1:].strip('][').split(', ')
                                print(type(guess2))
                            message=player2_socket.recv(1024).decode()
                            if message[0]=="g":
                                ec= message[1:].strip('][').split(', ')
                                a,b = submit(guess2,ec)
                                player2_socket.send(f"{a}{b}".encode())
    
                            # Wait for Player 2's turn
                            message = player2_socket.recv(1024).decode()
                            time.sleep(1)
                            # print(message)
                
                            # If Player 2 guessed, end the game
                            if message == "Guessed2":
                                pygame.display.set_caption('LOST!')
                                break
                                
                        if message=="Guessed2":
                            pygame.display.set_caption('LOST!')
                    else:
                        break
    except KeyboardInterrupt:
        print("Interrupted")
    except OSError as e:
    # Handle the error
        print(f"Error receiving data: {e}")
        
ready = env.ready
interactive_play(q_agent)
pygame.quit()
sys.exit()     

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Waiting for player 2 to connect...
Player 2 connected!
Player 2's secret number: 4354
4231, 5311, 1, 1
<class 'list'>
4231, 10168, 0, 1
<class 'list'>
4231, 2358, 0, 2
<class 'list'>
4231, 5142, 0, 3
<class 'list'>


SystemExit: 

C:\Users\Aishika Nandi\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
